----

## Store location analysis
#### Business Problem - An upcoming coffee supply chain company with direct access to coffee plantations wants to open its own stores in London. Before taking the first step to open its first store, the company would like to understand the market in general.

Approach - Foursquare API will be used to gather the location data and use it for analysis.
 - We will use the search query as 'Coffee' to procure the data for the current cafes in London
 - Additional search queries can be used to further study the reviews and ratings of the cafe to understand the general sentiment
 - Based on density of the cafes, a competitive landscape can be considered and leveraged to decide where to position the new cafe

In [7]:
#Importing necessary libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [8]:
# FourSquare API

CLIENT_ID = 'XRLQIDODGJUJ5YEJGXVVWJY1PPEAE130KUPIEXNNS1AYTTLU'
CLIENT_SECRET = 'PJJU2ZEXVUT4S1XBJDNQHP2OINEI01YAOCYBP5OTRH3NGOA2' 
VERSION = '20190301'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XRLQIDODGJUJ5YEJGXVVWJY1PPEAE130KUPIEXNNS1AYTTLU
CLIENT_SECRET:PJJU2ZEXVUT4S1XBJDNQHP2OINEI01YAOCYBP5OTRH3NGOA2


In [9]:
#Obtaining the required co-ordinates

address = 'London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.5073219 -0.1276474


In [49]:
search_query = 'Coffee'
radius = 2000

In [50]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XRLQIDODGJUJ5YEJGXVVWJY1PPEAE130KUPIEXNNS1AYTTLU&client_secret=PJJU2ZEXVUT4S1XBJDNQHP2OINEI01YAOCYBP5OTRH3NGOA2&ll=51.5073219,-0.1276474&v=20190301&query=Coffee&radius=2000&limit=30'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7cb45d949393001b2501e5'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'primary': True,
      'shortName': 'Coffee Shop'}],
    'hasPerk': False,
    'id': '4ac518edf964a520c1ac20e3',
    'location': {'address': '27 Monmouth St',
     'cc': 'GB',
     'city': 'Covent Garden',
     'country': 'United Kingdom',
     'distance': 780,
     'formattedAddress': ['27 Monmouth St',
      'Covent Garden',
      'Greater London',
      'WC2H 9EU',
      'United Kingdom'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.514314,
       'lng': -0.126824}],
     'lat': 51.514314,
     'lng': -0.126824,
     'neighborhood': 'Holborn and Covent Garden',
     'postalCode': 'WC2H 9EU',
     'state': 'Greater London'},
    'name': 'Monmouth Cof

In [52]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,4ac518edf964a520c1ac20e3,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,Monmouth Coffee Company,v-1585230945,NaN
1,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,4b4b47f5f964a520609626e3,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,Costa Coffee,v-1585230945,NaN
2,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,4b2630f7f964a5200a7924e3,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,Costa Coffee,v-1585230945,NaN
3,"[{'name': 'Coffee Shop', 'id': '4bf58dd8d48988...",False,551a93a2498ed90a2cc7e544,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,Department of Coffee and Social Affairs,v-1585230945,NaN
4,"[{'name': 'Café', 'id': '4bf58dd8d48988d16d941...",False,5883913551d19e062a758a77,173 Regent St,GB,London,United Kingdom,NaN,1009,"[173 Regent St, London, Greater London, W1B 4J...","[{'lat': 51.51219425144683, 'label': 'display'...",51.512194,-0.139929,NaN,W1B 4JQ,Greater London,Ralph's Coffee & Bar,v-1585230945,NaN


In [53]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,4ac518edf964a520c1ac20e3
1,Costa Coffee,Coffee Shop,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,4b4b47f5f964a520609626e3
2,Costa Coffee,Coffee Shop,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,4b2630f7f964a5200a7924e3
3,Department of Coffee and Social Affairs,Coffee Shop,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,551a93a2498ed90a2cc7e544
4,Ralph's Coffee & Bar,Café,173 Regent St,GB,London,United Kingdom,NaN,1009,"[173 Regent St, London, Greater London, W1B 4J...","[{'lat': 51.51219425144683, 'label': 'display'...",51.512194,-0.139929,NaN,W1B 4JQ,Greater London,5883913551d19e062a758a77


In [54]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,4ac518edf964a520c1ac20e3
1,Costa Coffee,Coffee Shop,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,4b4b47f5f964a520609626e3
2,Costa Coffee,Coffee Shop,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,4b2630f7f964a5200a7924e3
3,Department of Coffee and Social Affairs,Coffee Shop,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,551a93a2498ed90a2cc7e544
4,Ralph's Coffee & Bar,Café,173 Regent St,GB,London,United Kingdom,NaN,1009,"[173 Regent St, London, Greater London, W1B 4J...","[{'lat': 51.51219425144683, 'label': 'display'...",51.512194,-0.139929,NaN,W1B 4JQ,Greater London,5883913551d19e062a758a77
5,Costa Coffee,Coffee Shop,17 Embankment Pl,GB,London,United Kingdom,NaN,336,"[17 Embankment Pl, London, Greater London, WC2...","[{'lat': 51.50730858076348, 'label': 'display'...",51.507309,-0.122793,Embankmen,WC2N 6NN,Greater London,4ae815e0f964a52014ae21e3
6,Crosstown Doughnuts & Coffee,Donut Shop,4 Broadwick St,GB,London,United Kingdom,NaN,889,"[4 Broadwick St, London, Greater London, W1F 8...","[{'lat': 51.514042566300574, 'label': 'display...",51.514043,-0.134604,"Soho, London, Greater London",W1F 8HJ,Greater London,553e368d498ecddb31302a1c
7,Costa Coffee,Coffee Shop,15 Regent St,GB,London,United Kingdom,Jermyn St,462,"[15 Regent St (Jermyn St), London, Greater Lon...","[{'lat': 51.50879033505443, 'label': 'display'...",51.508790,-0.133887,NaN,SW1Y 4LR,Greater London,4cda944022bd721e533df047
8,Coffee Geek and Friends,Coffee Shop,Unit 22 Cardinal Place,GB,London,United Kingdom,6 Cathedral Walk,1444,"[Unit 22 Cardinal Place (6 Cathedral Walk), Lo...","[{'lat': 51.497637766011685, 'label': 'display...",51.497638,-0.141523,NaN,SW1E 5JH,Greater London,5565c24d498e2885d1b9a54e
9,Costa Coffee,Coffee Shop,13 New Row,GB,London,United Kingdom,NaN,447,"[13 New Row, London, Greater London, WC2N 4LF,...","[{'lat': 51.51115239980299, 'label': 'display'...",51.511152,-0.125678,NaN,WC2N 4LF,Greater London,4b24fad1f964a520f66a24e3


In [55]:
# Obtaining the ratings for the dataframe

rating=[]

for venue in dataframe_filtered['id']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    print(result)
    try:
        rating.append(result['response']['venue']['rating'])
    except:
        rating.append(0.0)

{'meta': {'errorType': 'quota_exceeded', 'code': 429, 'errorDetail': 'Quota exceeded', 'requestId': '5e7cb463c94979001b6abb11'}, 'response': {}}
{'meta': {'errorType': 'quota_exceeded', 'code': 429, 'errorDetail': 'Quota exceeded', 'requestId': '5e7cb3b3b4b684001ba661fc'}, 'response': {}}
{'meta': {'errorType': 'quota_exceeded', 'code': 429, 'errorDetail': 'Quota exceeded', 'requestId': '5e7cb34540a7ea001bbe2598'}, 'response': {}}
{'meta': {'errorType': 'quota_exceeded', 'code': 429, 'errorDetail': 'Quota exceeded', 'requestId': '5e7cb32baba297001b0485f4'}, 'response': {}}
{'meta': {'errorType': 'quota_exceeded', 'code': 429, 'errorDetail': 'Quota exceeded', 'requestId': '5e7cb50cdf2774001b71dc25'}, 'response': {}}
{'meta': {'errorType': 'quota_exceeded', 'code': 429, 'errorDetail': 'Quota exceeded', 'requestId': '5e7cb469216785001b5ab494'}, 'response': {}}
{'meta': {'errorType': 'quota_exceeded', 'code': 429, 'errorDetail': 'Quota exceeded', 'requestId': '5e7cb43aad1ab4001b671447'}, '

In [56]:
rating

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [57]:
# Adding the rating list procured to the dataframe

dataframe_filtered['rating']=rating
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,rating
0,Monmouth Coffee Company,Coffee Shop,27 Monmouth St,GB,Covent Garden,United Kingdom,NaN,780,"[27 Monmouth St, Covent Garden, Greater London...","[{'lat': 51.514314, 'label': 'display', 'lng':...",51.514314,-0.126824,Holborn and Covent Garden,WC2H 9EU,Greater London,4ac518edf964a520c1ac20e3,0.0
1,Costa Coffee,Coffee Shop,The Grand Building,GB,London,United Kingdom,Trafalgar Square,53,"[The Grand Building (Trafalgar Square), London...","[{'lat': 51.507644428472375, 'label': 'display...",51.507644,-0.127076,NaN,WC2N 5EJ,Greater London,4b4b47f5f964a520609626e3,0.0
2,Costa Coffee,Coffee Shop,200 Piccadilly,GB,London,United Kingdom,NaN,645,"[200 Piccadilly, London, Greater London, W1J 9...","[{'lat': 51.50896578159291, 'label': 'display'...",51.508966,-0.136586,NaN,W1J 9HU,Greater London,4b2630f7f964a5200a7924e3,0.0
3,Department of Coffee and Social Affairs,Coffee Shop,15 Sherwood St,GB,Soho,United Kingdom,Regent St Quadrant,694,"[15 Sherwood St (Regent St Quadrant), Soho, Gr...","[{'lat': 51.51087965006803, 'label': 'display'...",51.510880,-0.135874,"Piccadilly, London, Greater London",W1F 7ED,Greater London,551a93a2498ed90a2cc7e544,0.0
4,Ralph's Coffee & Bar,Café,173 Regent St,GB,London,United Kingdom,NaN,1009,"[173 Regent St, London, Greater London, W1B 4J...","[{'lat': 51.51219425144683, 'label': 'display'...",51.512194,-0.139929,NaN,W1B 4JQ,Greater London,5883913551d19e062a758a77,0.0


---